Researcher - TRISHUL CHOWDHURY

In [1]:
import os

In [3]:
import pandas as pd
data = pd.read_csv('amazon_200.csv', error_bad_lines=False);
data_text = data[['reviews.text']]
data_text['index'] = data_text.index
documents = data_text

C:\Users\Trishul Chowdhury\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Trishul Chowdhury\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [4]:
len(documents)

34660

In [5]:
documents[:5]

,reviews.text,index
0,This product so far has not disappointed. My c...,0
1,great for beginner or experienced person. Boug...,1
2,Inexpensive tablet for him to use and learn on...,2
3,I've had my Fire HD 8 two weeks now and I love...,3
4,I bought this for my grand daughter when she c...,4


In [6]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [7]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Trishul
[nltk_data]     Chowdhury\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [9]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [10]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['I', 'got', 'this', 'Kindle', 'as', 'a', 'Christmas', 'gift', 'for', 'a', 'friend.', "Couldn't", 'pass', 'up', 'the', 'Black', 'Friday', 'price', 'on', 'this', 'thing!', 'Works', 'great.', 'She', 'loves', 'it.', 'Success!']


 tokenized and lemmatized document: 
['kindl', 'christma', 'gift', 'friend', 'couldn', 'pass', 'black', 'friday', 'price', 'thing', 'work', 'great', 'love', 'success']


In [11]:
processed_docs = documents['reviews.text'].fillna('').astype(str).map(preprocess)

In [12]:
processed_docs[:10]

0    [product, disappoint, children, love, like, ab...
1     [great, beginn, experi, person, buy, gift, love]
2    [inexpens, tablet, learn, step, nabi, thrill, ...
3    [week, love, tablet, great, valu, prime, membe...
4    [buy, grand, daughter, come, visit, user, ente...
5    [amazon, inch, tablet, perfect, size, purchas,...
6    [great, read, nice, light, weight, price, poin...
7    [give, christma, gift, inlaw, husband, uncl, l...
8    [great, devic, read, book, like, link, borrow,...
9                    [love, order, book, read, reader]
Name: reviews.text, dtype: object

In [26]:
processed_docs

0        [product, disappoint, children, love, like, ab...
1         [great, beginn, experi, person, buy, gift, love]
2        [inexpens, tablet, learn, step, nabi, thrill, ...
3        [week, love, tablet, great, valu, prime, membe...
4        [buy, grand, daughter, come, visit, user, ente...
                               ...                        
34655    [appreci, faster, higher, charger, kindl, kind...
34656    [amazon, includ, charger, kindl, fact, charg, ...
34657    [love, kindl, disappoint, kindl, power, fast, ...
34658    [surpris, come, type, charg, cord, purchas, sp...
34659    [spite, fact, good, thing, amazon, anth, get, ...
Name: reviews.text, Length: 34660, dtype: object

In [13]:
## Bag of words 
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 abil
1 children
2 content
3 control
4 disappoint
5 eas
6 like
7 love
8 monitor
9 product
10 beginn


In [14]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [15]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(7, 1),
 (13, 1),
 (14, 1),
 (74, 1),
 (101, 1),
 (114, 1),
 (136, 1),
 (182, 1),
 (185, 1),
 (187, 1),
 (338, 1),
 (415, 1),
 (652, 1),
 (1450, 1)]

In [16]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 7 ("love") appears 1 time.
Word 13 ("gift") appears 1 time.
Word 14 ("great") appears 1 time.
Word 74 ("price") appears 1 time.
Word 101 ("christma") appears 1 time.
Word 114 ("kindl") appears 1 time.
Word 136 ("work") appears 1 time.
Word 182 ("couldn") appears 1 time.
Word 185 ("thing") appears 1 time.
Word 187 ("friend") appears 1 time.
Word 338 ("black") appears 1 time.
Word 415 ("friday") appears 1 time.
Word 652 ("pass") appears 1 time.
Word 1450 ("success") appears 1 time.


In [17]:
##TF IDF

In [18]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

In [19]:
corpus_tfidf = tfidf[bow_corpus]

In [20]:
from pprint import pprint

for doc in corpus_tfidf:
    print(doc)
    break

[(0, 0.35211031156819533), (1, 0.33520998250946255), (2, 0.3416445918662697), (3, 0.2610370507872522), (4, 0.3622034810513108), (5, 0.35852949058523387), (6, 0.16718112604987245), (7, 0.10388620564883835), (8, 0.5005859796176302), (9, 0.172418891517214)]


In [21]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [22]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())

[(0,
  '0.059*"echo" + 0.045*"alexa" + 0.036*"music" + 0.030*"love" + 0.028*"great" '
  '+ 0.027*"cabl" + 0.020*"home" + 0.018*"work" + 0.016*"play" + '
  '0.016*"question"'),
 (1,
  '0.039*"work" + 0.034*"tablet" + 0.024*"perfect" + 0.020*"firetv" + '
  '0.020*"purchas" + 0.017*"screen" + 0.016*"buy" + 0.016*"replac" + '
  '0.015*"kindl" + 0.014*"want"'),
 (2,
  '0.071*"product" + 0.063*"great" + 0.044*"recommend" + 0.041*"easi" + '
  '0.039*"purchas" + 0.037*"work" + 0.028*"love" + 0.021*"buy" + 0.020*"good" '
  '+ 0.020*"best"'),
 (3,
  '0.045*"love" + 0.034*"book" + 0.031*"buy" + 0.026*"read" + 0.024*"year" + '
  '0.021*"thing" + 0.019*"like" + 0.019*"play" + 0.016*"time" + 0.016*"list"'),
 (4,
  '0.080*"great" + 0.044*"tablet" + 0.036*"price" + 0.034*"good" + '
  '0.032*"amazon" + 0.026*"qualiti" + 0.021*"sound" + 0.018*"product" + '
  '0.015*"speaker" + 0.015*"money"'),
 (5,
  '0.035*"batteri" + 0.026*"life" + 0.016*"good" + 0.016*"need" + 0.016*"buy" '
  '+ 0.014*"long" + 0.014*

In [23]:
##Compute Model Perplexity and Coherence Score

In [28]:
id2word = corpora.Dictionary(processed_docs)

In [33]:
# Create Corpus
texts = processed_docs
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [29]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.36919537637695854


In [30]:
## Hyperparameter Tuning

In [36]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [37]:
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 540/540 [17:38:55<00:00, 117.66s/it]


In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [29]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.059*"echo" + 0.045*"alexa" + 0.036*"music" + 0.030*"love" + 0.029*"great" + 0.028*"cabl" + 0.020*"home" + 0.018*"work" + 0.016*"play" + 0.016*"question"
Topic: 1 
Words: 0.038*"work" + 0.034*"tablet" + 0.024*"perfect" + 0.020*"purchas" + 0.020*"firetv" + 0.017*"screen" + 0.016*"buy" + 0.016*"replac" + 0.015*"kindl" + 0.014*"want"
Topic: 2 
Words: 0.071*"product" + 0.063*"great" + 0.044*"recommend" + 0.041*"easi" + 0.039*"purchas" + 0.037*"work" + 0.028*"love" + 0.021*"buy" + 0.020*"good" + 0.020*"best"
Topic: 3 
Words: 0.045*"love" + 0.034*"book" + 0.031*"buy" + 0.026*"read" + 0.024*"year" + 0.021*"thing" + 0.019*"like" + 0.019*"play" + 0.016*"time" + 0.016*"list"
Topic: 4 
Words: 0.080*"great" + 0.044*"tablet" + 0.036*"price" + 0.034*"good" + 0.032*"amazon" + 0.026*"qualiti" + 0.021*"sound" + 0.018*"product" + 0.015*"speaker" + 0.015*"money"
Topic: 5 
Words: 0.035*"batteri" + 0.026*"life" + 0.016*"good" + 0.016*"buy" + 0.016*"need" + 0.014*"long" + 0.014*"alexa" + 0

In [2]:
#Running LDA using TF-IDF

In [30]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [32]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.020*"read" + 0.016*"kindl" + 0.016*"easi" + 0.015*"book" + 0.015*"love" + 0.014*"great" + 0.013*"good" + 0.012*"light" + 0.011*"voic" + 0.009*"work"
Topic: 1 Word: 0.015*"kid" + 0.014*"love" + 0.014*"great" + 0.011*"tablet" + 0.010*"easi" + 0.010*"time" + 0.010*"recommend" + 0.010*"product" + 0.009*"thank" + 0.008*"work"
Topic: 2 Word: 0.014*"firetv" + 0.011*"great" + 0.010*"pictur" + 0.009*"better" + 0.009*"work" + 0.008*"love" + 0.008*"good" + 0.008*"stick" + 0.008*"read" + 0.008*"best"
Topic: 3 Word: 0.036*"stick" + 0.015*"recommend" + 0.012*"love" + 0.012*"product" + 0.011*"great" + 0.011*"amazon" + 0.010*"purchas" + 0.010*"best" + 0.010*"cabl" + 0.010*"better"
Topic: 4 Word: 0.025*"firestick" + 0.020*"easi" + 0.018*"great" + 0.015*"tablet" + 0.012*"price" + 0.012*"love" + 0.011*"work" + 0.010*"good" + 0.010*"recommend" + 0.010*"batteri"
Topic: 5 Word: 0.014*"amazon" + 0.014*"great" + 0.011*"work" + 0.011*"easi" + 0.011*"roku" + 0.010*"like" + 0.009*"tablet" + 0.00

In [33]:
processed_docs[4310]

['kindl',
 'christma',
 'gift',
 'friend',
 'couldn',
 'pass',
 'black',
 'friday',
 'price',
 'thing',
 'work',
 'great',
 'love',
 'success']

In [34]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6681009531021118	 
Topic: 0.071*"product" + 0.063*"great" + 0.044*"recommend" + 0.041*"easi" + 0.039*"purchas" + 0.037*"work" + 0.028*"love" + 0.021*"buy" + 0.020*"good" + 0.020*"best"

Score: 0.15862774848937988	 
Topic: 0.038*"work" + 0.034*"tablet" + 0.024*"perfect" + 0.020*"purchas" + 0.020*"firetv" + 0.017*"screen" + 0.016*"buy" + 0.016*"replac" + 0.015*"kindl" + 0.014*"want"

Score: 0.12658506631851196	 
Topic: 0.035*"batteri" + 0.026*"life" + 0.016*"good" + 0.016*"buy" + 0.016*"need" + 0.014*"long" + 0.014*"alexa" + 0.013*"song" + 0.013*"love" + 0.012*"like"


In [35]:
#Performance evaluation by classifying sample document using LDA TF-IDF mode

In [36]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.6037395000457764	 
Topic: 0.020*"echo" + 0.017*"great" + 0.017*"alexa" + 0.015*"music" + 0.015*"love" + 0.014*"work" + 0.014*"enjoy" + 0.013*"product" + 0.010*"good" + 0.010*"speaker"

Score: 0.24526943266391754	 
Topic: 0.022*"love" + 0.020*"buy" + 0.015*"gift" + 0.015*"christma" + 0.014*"great" + 0.013*"product" + 0.011*"easi" + 0.011*"tablet" + 0.011*"roku" + 0.010*"friend"

Score: 0.10430815070867538	 
Topic: 0.014*"firetv" + 0.011*"great" + 0.010*"pictur" + 0.009*"better" + 0.009*"work" + 0.008*"love" + 0.008*"good" + 0.008*"stick" + 0.008*"read" + 0.008*"best"


In [38]:
unseen_document = 'I love the kindle. However I just hate their delivery process'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.32300981879234314	 Topic: 0.068*"read" + 0.062*"kindl" + 0.044*"light" + 0.032*"love" + 0.030*"book"
Score: 0.3205356299877167	 Topic: 0.071*"product" + 0.063*"great" + 0.044*"recommend" + 0.041*"easi" + 0.039*"purchas"
Score: 0.21639566123485565	 Topic: 0.052*"amazon" + 0.039*"prime" + 0.025*"easi" + 0.020*"remot" + 0.017*"movi"
Score: 0.0200138371437788	 Topic: 0.102*"love" + 0.041*"great" + 0.038*"kid" + 0.031*"buy" + 0.029*"easi"
Score: 0.020009679719805717	 Topic: 0.035*"batteri" + 0.026*"life" + 0.016*"good" + 0.016*"buy" + 0.016*"need"
Score: 0.02000967413187027	 Topic: 0.045*"love" + 0.034*"book" + 0.031*"buy" + 0.026*"read" + 0.024*"year"
Score: 0.020007513463497162	 Topic: 0.059*"echo" + 0.045*"alexa" + 0.036*"music" + 0.030*"love" + 0.029*"great"
Score: 0.02000698260962963	 Topic: 0.038*"work" + 0.034*"tablet" + 0.024*"perfect" + 0.020*"purchas" + 0.020*"firetv"
Score: 0.020005984231829643	 Topic: 0.080*"great" + 0.044*"tablet" + 0.036*"price" + 0.034*"good" + 0.032